In [11]:
import torch
from torch import nn
from torch import optim
import sys
import numpy as np
import random as rn
import pandas as pd
#BertModel
from pytorch_pretrained_bert import BertModel
from torchnlp.datasets import imdb_dataset
#from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output

import json
import collections
import pandas as pd
import transformers
from transformers import BertTokenizer

# This is a function that prints the number of trainable parameters 
# of a model.
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# This functions prints all parameters (and their gradients) of a model.
def print_parameters(model):
    for name, param in model.named_parameters():
        print(name)
        print(param.data)
        print(param.grad)

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

In [5]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout = 0.1):
        super(BertBinaryClassifier, self).__init()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self. sigmoid = nn.Sigmoid()
        
    def forward(self, tokens, masks = None):
        _, pooled_output = self.bert(tokens, attention_mask = masks, output_all_encoded_layers = False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [15]:
training_data = pd.read_json('WiC/train.jsonl', lines=True)

#Debug purposes - 
training_data.iloc[0:10]

,word,sentence1,sentence2,idx,label,start1,start2,end1,end2,version
0,place,Do you want to come over to my place later?,A political system with no place for the less ...,0,False,31,27,36,32,1.1
1,approach,Approach a task.,To approach the city.,1,False,0,3,8,11,1.1
2,run,Run rogue.,She ran 10 miles that day.,2,False,0,4,3,7,1.1
3,hold,The general ordered the colonel to hold his po...,Hold the taxi.,3,True,35,0,39,4,1.1
4,summer,We like to summer in the Mediterranean.,We summered in Kashmir.,4,True,11,3,17,11,1.1
5,head,His horse won by a head.,He is two heads taller than his little sister.,5,True,19,10,23,15,1.1
6,meet,The company agrees to meet the cost of any rep...,This proposal meets my requirements.,6,True,22,14,26,19,1.1
7,development,The organism has reached a crucial stage in it...,Our news team brings you the latest developments.,7,False,48,36,59,48,1.1
8,narrowness,The problem with achievement tests is the narr...,Frustrated by the narrowness of people's horiz...,8,True,42,18,52,28,1.1
9,act,The governor should act on the new energy bill.,Think before you act.,9,True,20,17,23,20,1.1
